In [2]:
import pandas as pd
import numpy as np
import math
import random

In [3]:
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random.uniform(-(2/n_inputs),2/n_inputs) for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random.uniform(-(2/n_hidden),2/n_hidden) for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [40]:
def activate(weights, inputs):
    activation = 0
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    activation += weights[-1]
    return activation

In [38]:
# Sigmoid
def transfer(activation):
    return 1.0 / (1.0 + math.exp(-activation))

In [39]:
# Sigmoid Derivative
def transfer_derivative(output):
        return output * (1.0 - output)

In [7]:
def forward_propagate(network, row):
        inputs = row
        for layer in network:
                new_inputs = []
                for neuron in layer:
                        activation = activate(neuron['weights'], inputs)
                        neuron['output'] = transfer(activation)
                        new_inputs.append(neuron['output'])
                inputs = new_inputs
        return inputs

In [8]:
def backward_propagate_error(network, expected):
    out_error = 0.0
    oLayer = network[-1]
    for i in range(len(network[-1])):
        neuron = oLayer[i]
        neuron['delta'] = (expected - neuron['output']) * transfer_derivative(neuron['output'])
        out_error = neuron['delta']
    for j in reversed(range(len(network[:-1]))):
        layer = network[j]
    for i, neuron in enumerate(layer):
        neuron['delta'] = (network[-1][0]['weights'][i] * out_error) * transfer_derivative(neuron['output'])

In [9]:
def update_weights(network, inputs, l_rate):
    for k in range(len(network)):
        layer = network[k]
        for j in range(len(layer)):
            neuron = layer[j]
            for i, input in enumerate(inputs):
                neuron['weights'][i] += l_rate * neuron['delta'] * input
            neuron['weights'][-1] += l_rate * neuron['delta']

In [10]:
#Update weights with momentum
def update_weightsM(network, inputs, l_rate):
    for k in range(len(network)):
        layer = network[k]
        for j in range(len(layer)):
            neuron = layer[j]
            prev_weight = neuron['weights'][:]
            for i, input in enumerate(inputs):
                if 'prev_weights' in neuron:
                    neuron['weights'][i] += l_rate * neuron['delta'] * input + (0.9 * (neuron['weights'][i] - neuron['prev_weights'][i]))
                else:
                    neuron['weights'][i] += l_rate * neuron['delta'] * input
            if 'prev_weights' in neuron:
                neuron['weights'][-1] += l_rate * neuron['delta'] + (0.9 * (neuron['weights'][-1] - neuron['prev_weights'][-1]))
            else:
                neuron['weights'][-1] += l_rate * neuron['delta']
            neuron['prev_weights'] = prev_weight

In [11]:
# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs[0]

In [46]:
def train_network(network, train, expected, l_rate, n_epoch):
    for epoch in range(n_epoch):
        sum_error = 0
        for i, row in enumerate(train):
            outputs = forward_propagate(network, row)
            sum_error += sum([(expected[i]-outputs[0])**2])
            backward_propagate_error(network, expected[i])
            update_weightsM(network, row, l_rate)
        RMSE = math.sqrt(sum_error/len(train))
        print('epoch=%d, lrate=%.3f, error=%.3f , rmse=%.6f' % (epoch, l_rate, sum_error, RMSE))

In [13]:
#Training the network with time based learning rate
def train_network(network, train, expected, l_rate, n_epoch):
    decay = l_rate / n_epoch
    for epoch in range(n_epoch):
        sum_error = 0
        l_rate = l_rate * 1/(1 + decay * epoch)
        for i, row in enumerate(train):
            outputs = forward_propagate(network, row)
            sum_error += sum([(expected[i]-outputs[0])**2])
            backward_propagate_error(network, expected[i])
            update_weightsM(network, row, l_rate)
        RMSE = math.sqrt(sum_error/len(train))
        print('epoch=%d, lrate=%.3f, error=%.3f , rmse=%.6f' % (epoch, l_rate, sum_error, RMSE))

In [14]:
# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = list()
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    return stats

In [15]:
# Rescale dataset columns to the range 0-1
def standardize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (0.8 * (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])) + 0.1

In [16]:
def destandardize_dataset(dataset, minmax):
     for row in dataset:
        for i in range(len(row)):
            row[i] = (((row[i] - 0.1) / 0.8) * (minmax[i][1] - minmax[i][0]) + minmax[i][0])

In [17]:
#Read csv file, containing the data
df = pd.read_csv('StudentData.csv')
#Get the dataset values in an array
dataset = df.values
#Return an array containing the minimum and maximum values for each column in the dataset
minmax = dataset_minmax(dataset)
#Standardize the dataset
standardize_dataset(dataset, minmax)

In [18]:
split = math.ceil(0.6 * len(dataset))
split1 = math.ceil(0.8 * len(dataset))

#Validation set
valid = dataset[split1:,:]
validX, validY = valid[:, :-1], valid[:, -1]

#Test set
test = dataset[split:split1,:]
testX, testY = test[:, :-1], test[:, -1]

#Train set
train =  dataset[:split,:]
trainX, trainY = train[:, :-1], train[:, -1]

In [43]:
network = initialize_network(trainX.shape[1], 5, 1)

In [47]:
l_rate = 0.1
n_epoch = 1000
train_network(network, trainX, trainY, l_rate, n_epoch)

epoch=0, lrate=0.100, error=1.744 , rmse=0.044701
epoch=1, lrate=0.100, error=1.752 , rmse=0.044794
epoch=2, lrate=0.100, error=1.751 , rmse=0.044787
epoch=3, lrate=0.100, error=1.751 , rmse=0.044780
epoch=4, lrate=0.100, error=1.750 , rmse=0.044772
epoch=5, lrate=0.100, error=1.749 , rmse=0.044764
epoch=6, lrate=0.100, error=1.749 , rmse=0.044756
epoch=7, lrate=0.100, error=1.748 , rmse=0.044748
epoch=8, lrate=0.100, error=1.747 , rmse=0.044739
epoch=9, lrate=0.100, error=1.747 , rmse=0.044731
epoch=10, lrate=0.100, error=1.746 , rmse=0.044722
epoch=11, lrate=0.100, error=1.745 , rmse=0.044713
epoch=12, lrate=0.100, error=1.745 , rmse=0.044704
epoch=13, lrate=0.100, error=1.744 , rmse=0.044695
epoch=14, lrate=0.100, error=1.743 , rmse=0.044686
epoch=15, lrate=0.100, error=1.742 , rmse=0.044676
epoch=16, lrate=0.100, error=1.742 , rmse=0.044667
epoch=17, lrate=0.100, error=1.741 , rmse=0.044658
epoch=18, lrate=0.100, error=1.740 , rmse=0.044648
epoch=19, lrate=0.100, error=1.740 , rmse

epoch=160, lrate=0.100, error=1.626 , rmse=0.043157
epoch=161, lrate=0.100, error=1.625 , rmse=0.043149
epoch=162, lrate=0.100, error=1.625 , rmse=0.043142
epoch=163, lrate=0.100, error=1.624 , rmse=0.043134
epoch=164, lrate=0.100, error=1.624 , rmse=0.043127
epoch=165, lrate=0.100, error=1.623 , rmse=0.043120
epoch=166, lrate=0.100, error=1.623 , rmse=0.043112
epoch=167, lrate=0.100, error=1.622 , rmse=0.043105
epoch=168, lrate=0.100, error=1.622 , rmse=0.043098
epoch=169, lrate=0.100, error=1.621 , rmse=0.043091
epoch=170, lrate=0.100, error=1.620 , rmse=0.043084
epoch=171, lrate=0.100, error=1.620 , rmse=0.043077
epoch=172, lrate=0.100, error=1.619 , rmse=0.043070
epoch=173, lrate=0.100, error=1.619 , rmse=0.043063
epoch=174, lrate=0.100, error=1.618 , rmse=0.043056
epoch=175, lrate=0.100, error=1.618 , rmse=0.043049
epoch=176, lrate=0.100, error=1.617 , rmse=0.043042
epoch=177, lrate=0.100, error=1.617 , rmse=0.043035
epoch=178, lrate=0.100, error=1.616 , rmse=0.043028
epoch=179, l

epoch=317, lrate=0.100, error=1.569 , rmse=0.042398
epoch=318, lrate=0.100, error=1.569 , rmse=0.042395
epoch=319, lrate=0.100, error=1.569 , rmse=0.042392
epoch=320, lrate=0.100, error=1.569 , rmse=0.042389
epoch=321, lrate=0.100, error=1.568 , rmse=0.042386
epoch=322, lrate=0.100, error=1.568 , rmse=0.042383
epoch=323, lrate=0.100, error=1.568 , rmse=0.042379
epoch=324, lrate=0.100, error=1.568 , rmse=0.042376
epoch=325, lrate=0.100, error=1.567 , rmse=0.042373
epoch=326, lrate=0.100, error=1.567 , rmse=0.042370
epoch=327, lrate=0.100, error=1.567 , rmse=0.042367
epoch=328, lrate=0.100, error=1.567 , rmse=0.042364
epoch=329, lrate=0.100, error=1.567 , rmse=0.042361
epoch=330, lrate=0.100, error=1.566 , rmse=0.042358
epoch=331, lrate=0.100, error=1.566 , rmse=0.042355
epoch=332, lrate=0.100, error=1.566 , rmse=0.042352
epoch=333, lrate=0.100, error=1.566 , rmse=0.042349
epoch=334, lrate=0.100, error=1.565 , rmse=0.042346
epoch=335, lrate=0.100, error=1.565 , rmse=0.042343
epoch=336, l

epoch=476, lrate=0.100, error=1.541 , rmse=0.042007
epoch=477, lrate=0.100, error=1.540 , rmse=0.042006
epoch=478, lrate=0.100, error=1.540 , rmse=0.042004
epoch=479, lrate=0.100, error=1.540 , rmse=0.042002
epoch=480, lrate=0.100, error=1.540 , rmse=0.042000
epoch=481, lrate=0.100, error=1.540 , rmse=0.041998
epoch=482, lrate=0.100, error=1.540 , rmse=0.041996
epoch=483, lrate=0.100, error=1.540 , rmse=0.041995
epoch=484, lrate=0.100, error=1.539 , rmse=0.041993
epoch=485, lrate=0.100, error=1.539 , rmse=0.041991
epoch=486, lrate=0.100, error=1.539 , rmse=0.041989
epoch=487, lrate=0.100, error=1.539 , rmse=0.041987
epoch=488, lrate=0.100, error=1.539 , rmse=0.041986
epoch=489, lrate=0.100, error=1.539 , rmse=0.041984
epoch=490, lrate=0.100, error=1.539 , rmse=0.041982
epoch=491, lrate=0.100, error=1.539 , rmse=0.041980
epoch=492, lrate=0.100, error=1.538 , rmse=0.041979
epoch=493, lrate=0.100, error=1.538 , rmse=0.041977
epoch=494, lrate=0.100, error=1.538 , rmse=0.041975
epoch=495, l

epoch=635, lrate=0.100, error=1.522 , rmse=0.041757
epoch=636, lrate=0.100, error=1.522 , rmse=0.041755
epoch=637, lrate=0.100, error=1.522 , rmse=0.041754
epoch=638, lrate=0.100, error=1.522 , rmse=0.041752
epoch=639, lrate=0.100, error=1.522 , rmse=0.041751
epoch=640, lrate=0.100, error=1.522 , rmse=0.041750
epoch=641, lrate=0.100, error=1.522 , rmse=0.041748
epoch=642, lrate=0.100, error=1.521 , rmse=0.041747
epoch=643, lrate=0.100, error=1.521 , rmse=0.041745
epoch=644, lrate=0.100, error=1.521 , rmse=0.041744
epoch=645, lrate=0.100, error=1.521 , rmse=0.041743
epoch=646, lrate=0.100, error=1.521 , rmse=0.041741
epoch=647, lrate=0.100, error=1.521 , rmse=0.041740
epoch=648, lrate=0.100, error=1.521 , rmse=0.041738
epoch=649, lrate=0.100, error=1.521 , rmse=0.041737
epoch=650, lrate=0.100, error=1.521 , rmse=0.041736
epoch=651, lrate=0.100, error=1.521 , rmse=0.041734
epoch=652, lrate=0.100, error=1.520 , rmse=0.041733
epoch=653, lrate=0.100, error=1.520 , rmse=0.041731
epoch=654, l

epoch=794, lrate=0.100, error=1.507 , rmse=0.041550
epoch=795, lrate=0.100, error=1.507 , rmse=0.041549
epoch=796, lrate=0.100, error=1.507 , rmse=0.041547
epoch=797, lrate=0.100, error=1.507 , rmse=0.041546
epoch=798, lrate=0.100, error=1.507 , rmse=0.041545
epoch=799, lrate=0.100, error=1.507 , rmse=0.041544
epoch=800, lrate=0.100, error=1.507 , rmse=0.041543
epoch=801, lrate=0.100, error=1.507 , rmse=0.041542
epoch=802, lrate=0.100, error=1.506 , rmse=0.041540
epoch=803, lrate=0.100, error=1.506 , rmse=0.041539
epoch=804, lrate=0.100, error=1.506 , rmse=0.041538
epoch=805, lrate=0.100, error=1.506 , rmse=0.041537
epoch=806, lrate=0.100, error=1.506 , rmse=0.041536
epoch=807, lrate=0.100, error=1.506 , rmse=0.041535
epoch=808, lrate=0.100, error=1.506 , rmse=0.041533
epoch=809, lrate=0.100, error=1.506 , rmse=0.041532
epoch=810, lrate=0.100, error=1.506 , rmse=0.041531
epoch=811, lrate=0.100, error=1.506 , rmse=0.041530
epoch=812, lrate=0.100, error=1.506 , rmse=0.041529
epoch=813, l

epoch=952, lrate=0.100, error=1.496 , rmse=0.041395
epoch=953, lrate=0.100, error=1.496 , rmse=0.041394
epoch=954, lrate=0.100, error=1.496 , rmse=0.041393
epoch=955, lrate=0.100, error=1.496 , rmse=0.041393
epoch=956, lrate=0.100, error=1.496 , rmse=0.041392
epoch=957, lrate=0.100, error=1.496 , rmse=0.041391
epoch=958, lrate=0.100, error=1.496 , rmse=0.041391
epoch=959, lrate=0.100, error=1.496 , rmse=0.041390
epoch=960, lrate=0.100, error=1.496 , rmse=0.041389
epoch=961, lrate=0.100, error=1.495 , rmse=0.041388
epoch=962, lrate=0.100, error=1.495 , rmse=0.041388
epoch=963, lrate=0.100, error=1.495 , rmse=0.041387
epoch=964, lrate=0.100, error=1.495 , rmse=0.041386
epoch=965, lrate=0.100, error=1.495 , rmse=0.041386
epoch=966, lrate=0.100, error=1.495 , rmse=0.041385
epoch=967, lrate=0.100, error=1.495 , rmse=0.041384
epoch=968, lrate=0.100, error=1.495 , rmse=0.041384
epoch=969, lrate=0.100, error=1.495 , rmse=0.041383
epoch=970, lrate=0.100, error=1.495 , rmse=0.041382
epoch=971, l

In [35]:
prediction = []
sum_error=0

print('Expected'.rjust(15), 'Predicted'.rjust(18))
for i, row in enumerate(testX):
    prediction.append(predict(network, row))
    print(testY[i], prediction[i])
    
for i, row in enumerate(testY):
    sum_error += sum([(row-prediction[i])**2])
    RMSE = math.sqrt(sum_error/len(testX))
print('error=%.3f , rmse=%.6f' % (sum_error, RMSE))

       Expected          Predicted
0.5720496894409938 0.0
0.5968944099378881 0.0
0.7608695652173912 0.0
0.6813664596273291 0.0
0.43788819875776397 0.0
0.184472049689441 0.0
0.25900621118012424 0.0
0.21428571428571427 0.0
0.17453416149068324 0.0
0.21925465838509317 0.0
0.1248447204968944 0.0
0.184472049689441 0.0
0.184472049689441 0.0
0.33354037267080744 0.0
0.13975155279503107 0.0
0.1546583850931677 0.0
0.14472049689440994 0.0
0.4329192546583851 0.0
0.5770186335403726 0.0
0.5372670807453417 0.0
0.3285714285714285 0.0
0.7906832298136646 0.0
0.8006211180124223 0.0
0.5124223602484472 0.0
0.4031055900621118 0.0
0.775776397515528 0.0
0.11490683229813665 0.0
0.22919254658385094 0.0
0.11490683229813665 0.0
0.24906832298136647 0.0
0.1894409937888199 0.0
0.30869565217391304 0.0
0.35838509316770184 0.0
0.8354037267080745 0.0
0.48757763975155277 0.0
0.5472049689440993 0.0
0.22422360248447204 0.0
0.30869565217391304 0.0
0.29378881987577643 0.0
0.24409937888198757 0.0
0.2639751552795031 0.0
0.72608